<a href="https://colab.research.google.com/github/DerJFK/codewars/blob/main/math_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://www.codewars.com/kata/52a78825cdfc2cfc87000005
# https://github.com/lostleaf/codewars/blob/master/evaluate-mathematical-expression.py

import re
import numpy as np
import pandas as pd
import networkx as nx # https://networkx.org/documentation/latest/tutorial.html

In [2]:
ops = {
    '*' : lambda x,y: x * y,
    '/' : lambda x,y: x / y,
    '+' : lambda x,y: x + y,
    '-' : lambda x,y: x - y
}

In [3]:
expr = (
        ("1 + 1", 2),
        ("8/16", 0.5),
        ("3 -(-1)", 4),
        ("2 + -2", 0),
        ("10- 2- -5", 13),
        ("(((10)))", 10),
        ("3 * 5", 15),
        ("-7 * -(6 / 3)", 14)
        )

In [4]:
regex = {
    'NB' : re.compile(r"\d+(\.\d*)?"),
    'PL' : re.compile(r"\("),
    'PR' : re.compile(r"\)"),
    'OP' : re.compile(r"[+\-*/]")
}
regex = pd.Series(regex)

def nextToken(expr):
  res = regex.apply(lambda x: x.search(expr))
  idx = res.apply(lambda x: x.start() if x is not None else np.inf).idxmin()
  return res[idx], idx

In [5]:
class BaseExpr:
  def __init__(self, expr, start, end, leftExpr=None, rightExpr=None):
    self.left = leftExpr
    self.right = rightExpr
    self.start = start
    self.end = end
    self.expr = expr

  def __str__(self):
    return self.expr[self.start:self.end]

  def merge(self):
    return False

class NBExpr(BaseExpr):
  def __init__(self, expr, start, end, leftExpr=None, rightExpr=None):
    super().__init__(expr, start, end, leftExpr, rightExpr)
    self.z = float(self.expr[self.start:self.end])

class PExpr(BaseExpr):
  pass
  
class MultDivExpr(BaseExpr):
  pass

class AddSubExpr(BaseExpr):
  
  def merge(self):
    if self.right is not None \
      and self.left is not None \
      and (type(self.left) is AddSubExpr or type(self.left) is PExpr) \
      and type(self.right) is NBExpr:

      print(self.expr[self.start:self.right.end])

      self.right.z = float(self.expr[self.start:self.right.end])
      self.left.right = self.right
      self.right.start = self.start

      return True
    
    if self.left is None \
      and self.right is not None \
      and type(self.right) is NBExpr:

      print(self.expr[self.start:self.right.end])
      self.right.z = float(self.expr[self.start:self.right.end])
      self.right.start = self.start
      self.right.left = None

      return True

    return False


nodeCLS = {
    'NB' : NBExpr,
    'PL' : PExpr,
    'PR' : PExpr,
    '*' : MultDivExpr,
    '/' : MultDivExpr,
    '+': AddSubExpr,
    '-' : AddSubExpr
}

def appendNode(node, nextNode):
  if node is None:
    return nextNode
  else:
    node.right = nextNode
    nextNode.left = node
    return nextNode

def iterNodes(node):
  n = node
  while True:
    if n is None:
      break
    yield n
    n = n.right


In [6]:
e = '-2 * (3 + -5)'
node = None
start = end = 0
while True:
  m, key = nextToken(e[start:])
  if m is None: break

  end = start + m.end()
  start = start + m.start()

  if key == 'OP':
    op = e[start:end]
    node = appendNode(node, nodeCLS[op](e, start, end))
  else:
    node = appendNode(node, nodeCLS[key](e, start, end))
  start = end

endNode = node
while node.left is not None:
  node = node.left
startNode = node

In [7]:
def reduceSign(expr):

  if len(expr) > 1:
    x, y = expr[:2]
    if x == '-' and type(y) is float:

      return [-1 * y] + expr[2:] 

  i = 2
  while i < len(expr):
    
    x, y, z = expr[i-2:i+1]
    if (x == '+' or x == '-' or x == '(') \
      and y == '-' and type(z) is float:

      return expr[:i-1] + [-1 * z] + expr[i+1:] 

    i += 1
  
  return expr

def reduceMultDiv(expr):
  i = 3
  while i < len(expr):
    x,op,y = expr[i-3:i]
    if type(x) is float \
      and (op == '*' or op=='/') \
      and type(y) is float:

      print(x,op,y)
      return expr[:i-3] + [ops[op](x,y)] + expr[i:]
    i += 1

  return expr


In [8]:
e = "-7 * -(6 / 3) + 10 * ((3 + -5) * (3/(5+1)))"

start = end = 0
i, j = 0, 0
expr = []

while True:
  m, key = nextToken(e[start:])
  if m is None: break

  end = start + m.end()
  start = start + m.start()
  
  if key == 'NB':
    expr.append(float(e[start:end]))
  else:
    expr.append(e[start:end])

  start = end


m, n = 0, 1
while m < n:
  n = len(expr)
  expr = reduceSign(expr)
  m = len(expr)

m, n = 0, 1
while m < n:
  n = len(expr)
  expr = reduceMultDiv(expr)
  m = len(expr)

expr



6.0 / 3.0


[-7.0,
 '*',
 '-',
 '(',
 2.0,
 ')',
 '+',
 10.0,
 '*',
 '(',
 '(',
 3.0,
 '+',
 -5.0,
 ')',
 '*',
 '(',
 3.0,
 '/',
 '(',
 5.0,
 '+',
 1.0,
 ')',
 ')',
 ')']